### Environment Setup

In [1]:
import datetime

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import MobileNetV2

%load_ext tensorboard

### Ingest, Image Preprocessing and Augmentation

In [2]:
train_data_dir = "../../Google Drive/My Drive/498/Project/mushie_image_data/"
num_classes = 2
img_width, img_height = 224, 224
classes = ['poisonous', 'edible']
batch_size = 32

# NOTE: our model will have a single output node
# This means that an output of '0' means a prediction of poisonous,
# And an output of '1' means a prediction of edible
# To flip this, change the order of the classes above

In [3]:
# Define image augmentation methods here
# As well as the train/validation split (thanks Keras for adding that feature!)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

print("Training set:")
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    classes = classes,
    shuffle=True,
    class_mode='binary',
    subset='training')

print("Validation set:")
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    classes = classes,
    class_mode='binary',
    subset='validation')

Training set:
Found 7135 images belonging to 2 classes.
Validation set:
Found 1782 images belonging to 2 classes.


In [4]:
### Modeling Setup

In [5]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [7]:
print("These layers will be set to untrainable: ")
for layer in base_model.layers[:-11]:
    layer.trainable = False
    print(layer.name)

These layers will be set to untrainable: 
input_1
Conv1
bn_Conv1
Conv1_relu
expanded_conv_depthwise
expanded_conv_depthwise_BN
expanded_conv_depthwise_relu
expanded_conv_project
expanded_conv_project_BN
block_1_expand
block_1_expand_BN
block_1_expand_relu
block_1_pad
block_1_depthwise
block_1_depthwise_BN
block_1_depthwise_relu
block_1_project
block_1_project_BN
block_2_expand
block_2_expand_BN
block_2_expand_relu
block_2_depthwise
block_2_depthwise_BN
block_2_depthwise_relu
block_2_project
block_2_project_BN
block_2_add
block_3_expand
block_3_expand_BN
block_3_expand_relu
block_3_pad
block_3_depthwise
block_3_depthwise_BN
block_3_depthwise_relu
block_3_project
block_3_project_BN
block_4_expand
block_4_expand_BN
block_4_expand_relu
block_4_depthwise
block_4_depthwise_BN
block_4_depthwise_relu
block_4_project
block_4_project_BN
block_4_add
block_5_expand
block_5_expand_BN
block_5_expand_relu
block_5_depthwise
block_5_depthwise_BN
block_5_depthwise_relu
block_5_project
block_5_project_BN

In [14]:
# Now we set the highest convolution block to trainable
# But make sure to not allow any BatchNorm layers be trainable
# https://keras.io/guides/transfer_learning/#finetuning
for layer in base_model.layers[-11:]:
    if "BN" not in layer.name and "bn" not in layer.name:
        layer.trainable = True
        print("\033[93m Trainable: ", layer.name, "\033[0m")
    else:
        layer.trainable = False
        print("Untrainable: ", layer.name)

 Trainable:  block_16_expand 
Untrainable:  block_16_expand_BN
 Trainable:  block_16_expand_relu 
 Trainable:  block_16_depthwise 
Untrainable:  block_16_depthwise_BN
 Trainable:  block_16_depthwise_relu 
 Trainable:  block_16_project 
Untrainable:  block_16_project_BN
 Trainable:  Conv_1 
Untrainable:  Conv_1_bn
 Trainable:  out_relu 


In [9]:
# Add in new top layers
# With a sigmoid output node (so we can do binary classification)
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 880,321
Non-trainable params: 1,378,944
_________________________________________________________________


### Tensorboard Evaluation

In [10]:
# To launch tensorboard, run this cell
# Enable auto-reloading in the settings menu (it looks like a gear)
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

%tensorboard --logdir logs

### Training

In [11]:
save_checkpoints = tf.keras.callbacks.ModelCheckpoint(
    filepath='./tmp/checkpoint',
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

In [ ]:
#enable early stopping
es = tf.keras.callbacks.EarlyStopping(
                                 monitor='val_acc',
                                 patience=5,
                                 mode='auto',
                                 baseline=None,
                                 restore_best_weights=True
                                )

In [12]:
#compile and train the model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adagrad(),
              metrics=['acc', 'AUC'])

In [13]:
EPOCHS = 30
model.fit(train_generator,
          epochs = EPOCHS,
          steps_per_epoch = train_generator.samples // batch_size,
          validation_data=validation_generator,
          validation_steps = validation_generator.samples // batch_size,
          callbacks=[tensorboard_cb, save_checkpoints]
         )

Epoch 1/30
151/222 [===================>..........] - ETA: 1:03 - loss: 0.7422 - acc: 0.5739

/Users/christophrico/.local/share/virtualenvs/mushie-classifier-hjQZkQ93/lib/python3.8/site-packages/PIL/Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


222/222 [==============================] - 257s 1s/step - loss: 0.7120 - acc: 0.5938 - val_loss: 0.6317 - val_acc: 0.6358
Epoch 2/30
222/222 [==============================] - 256s 1s/step - loss: 0.5875 - acc: 0.6863 - val_loss: 0.6230 - val_acc: 0.6602
Epoch 3/30
222/222 [==============================] - 252s 1s/step - loss: 0.5614 - acc: 0.7026 - val_loss: 0.6340 - val_acc: 0.6438
Epoch 4/30
222/222 [==============================] - 260s 1s/step - loss: 0.5510 - acc: 0.7123 - val_loss: 0.6114 - val_acc: 0.6665
Epoch 5/30
222/222 [==============================] - 265s 1s/step - loss: 0.5176 - acc: 0.7345 - val_loss: 0.6425 - val_acc: 0.6409
Epoch 6/30
222/222 [==============================] - 265s 1s/step - loss: 0.5073 - acc: 0.7518 - val_loss: 0.6047 - val_acc: 0.6722
Epoch 7/30
222/222 [==============================] - 244s 1s/step - loss: 0.5046 - acc: 0.7562 - val_loss: 0.6282 - val_acc: 0.6506
Epoch 8/30
222/222 [==============================] - 245s 1s/step - loss: 0.485

In [15]:
model.save('saved_model/mushie_model_partially_trained.h5')

### Fine-Tuning

In [16]:
# Unfreeze all the layers to trainable to do some fine-tuning
for layer in model.layers[0].layers:
# But make sure to not allow any BatchNorm layers be trainable
# https://keras.io/guides/transfer_learning/#finetuning
    if "BN" not in layer.name and "bn" not in layer.name:
        layer.trainable = True
        print("Trainable: ", layer.name)
    else:
        layer.trainable = False
        print("\033[93m Untrainable: ", layer.name, "\033[0m")

Trainable:  input_1
Trainable:  Conv1
 Untrainable:  bn_Conv1 
Trainable:  Conv1_relu
Trainable:  expanded_conv_depthwise
 Untrainable:  expanded_conv_depthwise_BN 
Trainable:  expanded_conv_depthwise_relu
Trainable:  expanded_conv_project
 Untrainable:  expanded_conv_project_BN 
Trainable:  block_1_expand
 Untrainable:  block_1_expand_BN 
Trainable:  block_1_expand_relu
Trainable:  block_1_pad
Trainable:  block_1_depthwise
 Untrainable:  block_1_depthwise_BN 
Trainable:  block_1_depthwise_relu
Trainable:  block_1_project
 Untrainable:  block_1_project_BN 
Trainable:  block_2_expand
 Untrainable:  block_2_expand_BN 
Trainable:  block_2_expand_relu
Trainable:  block_2_depthwise
 Untrainable:  block_2_depthwise_BN 
Trainable:  block_2_depthwise_relu
Trainable:  block_2_project
 Untrainable:  block_2_project_BN 
Trainable:  block_2_add
Trainable:  block_3_expand
 Untrainable:  block_3_expand_BN 
Trainable:  block_3_expand_relu
Trainable:  block_3_pad
Trainable:  block_3_depthwise
 Untrain

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 2,191,041
Non-trainable params: 68,224
_________________________________________________________________


In [19]:
# Compile and train the model to learn at an extremely slow rate
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adagrad(1e-5),
              metrics=['acc', 'AUC'])

In [20]:
EPOCHS = 30
model.fit(train_generator,
          epochs = EPOCHS,
          steps_per_epoch = train_generator.samples // batch_size,
          validation_data=validation_generator,
          validation_steps = validation_generator.samples // batch_size
         )

Epoch 1/30
222/222 [==============================] - 581s 3s/step - loss: 0.2771 - acc: 0.8960 - auc: 0.9618 - val_loss: 0.6140 - val_acc: 0.6949 - val_auc: 0.7626
Epoch 2/30
222/222 [==============================] - 547s 2s/step - loss: 0.2673 - acc: 0.9024 - auc: 0.9642 - val_loss: 0.6286 - val_acc: 0.6801 - val_auc: 0.7512
Epoch 3/30
222/222 [==============================] - 542s 2s/step - loss: 0.2728 - acc: 0.8913 - auc: 0.9627 - val_loss: 0.6230 - val_acc: 0.6972 - val_auc: 0.7588
Epoch 4/30
222/222 [==============================] - 544s 2s/step - loss: 0.2752 - acc: 0.8909 - auc: 0.9606 - val_loss: 0.6313 - val_acc: 0.6915 - val_auc: 0.7584
Epoch 5/30
222/222 [==============================] - 545s 2s/step - loss: 0.2632 - acc: 0.9006 - auc: 0.9650 - val_loss: 0.6171 - val_acc: 0.6943 - val_auc: 0.7596
Epoch 6/30
222/222 [==============================] - 547s 2s/step - loss: 0.2682 - acc: 0.8993 - auc: 0.9633 - val_loss: 0.6374 - val_acc: 0.6801 - val_auc: 0.7531
Epoch 7/30

KeyboardInterrupt: 

In [21]:
model.save('saved_model/mushie_model_partially_trained.h5')